# General Imports, Settings, and Methods

In [ ]:
import pandas as pd
import requests
import datetime
import math
from google.cloud import bigquery
import scipy.stats  as stats
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
%matplotlib inline

## Formatting and the like

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as mtick
import matplotlib
from decimal import Decimal
pd.options.mode.chained_assignment = None  # default='warn'
style.use('ggplot')
# plt.tight_layout()
# color_map = plt.get_cmap('tab20b')
# colors = [  plt.cm.tab20b(x) for x in [0, 4, 8, 12, 16, 1, 5, 9, 13, 17, 2, 6, 10, 14, 18] ]
colors = [ matplotlib.colors.to_hex(plt.cm.Dark2(x)) for x in [0,1,2,3,4,5,6,7,0,1,2,3,4,5,6,7,0,1,2,3,4,5,6,7,0,1,2,3,4,5,6,7,0,1,2,3,4,5,6,7,0,1,2,3,4,5,6,7,0,1,2,3,4,5,6,7,0,1,2,3,4,5,6,7,0,1,2,3,4,5,6,7,0,1,2,3,4,5,6,7]]
plt.rcParams['axes.facecolor']='#ebeae8'

## Helper methods

In [ ]:
def get_google_sheet_df(sheet_id: str, sheet_name: str) -> pd.DataFrame:
    """
    Get data from a Google Sheet document.
    """

    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={requests.utils.quote(sheet_name)}"
    print(f"Getting: {url}")
    return pd.read_csv(url).dropna()

In [ ]:
from typing import Tuple

def printable_p(p: float) -> str:
    """
    Return a more human readable p-value.
    """
    if p <= 0.006:
        return f"{p:0.1e}"
    return f"{p:0.2f}"

def human_col_info(col_name: str) -> Tuple[str, str]:
    """
    Return more human-redable representations of columns if they exist.
    """
    units = col_units[col_name] if col_name in col_units else r'$Unkn$'
    name = col_names[col_name] if col_name in col_names else col_name
    return name, units

def normalize_to_mean(col: pd.Series) -> pd.Series:
    """
    Return a column (as a pandas.Series) with values "normalized" with a
    mean of 0.0 and a standard deviation of 1.0
    """
    return (col - col.mean()) / col.std()

def plot_two_ts(df: pd.DataFrame, col_a_name: str, col_b_name: str,
             title: str=None, norm: bool=False, subtitle: str="",
             rolling: int=None, corr: bool=True, figsize: Tuple[int, int]=(9,6)
             ) -> None:
    """
    Convenience function to plot two columns with dates (aka time-series) and
    possibly correlation information.
    """
    fig, ax = plt.subplots(figsize=figsize)
    # Get the minimum and maximum dates
    mindate, maxdate = df['date'].min(),df['date'].max()
    # Make the x-axis "tight" (no empty space before or after the actual data)
    ax.set_xlim(mindate,maxdate)
    # Make the mininum and maximum date "pretty"
    mindate_str, maxdate_str = mindate.strftime('%b %d, %Y'), maxdate.strftime('%b %d, %Y')
    # Get human readable column name and units
    name_a, units_a = human_col_info(col_a_name)
    name_b, units_b = human_col_info(col_b_name)
    # If the units are different, use two axes so that scale is (likely) less
    # of an issue.
    if not norm and units_a != units_b:
        ax2 = ax.twinx()
        ax2.set_ylabel(units_b)
    else:
        ax2 = ax
        pass
    # Create a temporary DataFrame so that we can change things if needed
    # without affecting the original DataFrame
    tmp_df = df[[col_a_name, col_b_name, 'date']]
    # Normalize the data, if we're supposed to.
    if norm:
        tmp_df[col_a_name] = normalize_to_mean(tmp_df[col_a_name])
        tmp_df[col_b_name] = normalize_to_mean(tmp_df[col_b_name])
        pass
    # Use a rolling mean if we're supposed to.
    if rolling:
        tmp_df[col_a_name] = tmp_df[col_a_name].rolling(rolling).mean()
        tmp_df[col_b_name] = tmp_df[col_b_name].rolling(rolling).mean()
        pass
    # Plot the lines
    tmp_df.plot(x='date', y=col_a_name, ax=ax, lw=1.5, alpha=0.5, color=colors[0], legend=False)
    tmp_df.plot(x='date', y=col_b_name, ax=ax2, lw=1.5, alpha=0.5, color=colors[1], legend=False)
    # Create a temp DataFrame with only the two columns we just plotted and
    # remove any rows witn NaNs in them (the stats package Spearman/Pearson
    # functions don't like NaNs, but the DataFrame versions don't return a
    # p-value, so we're going to use the stats version and need to remove
    # all the NaNs)
    tmp_df.dropna(inplace=True)
    # Get to Numpy arrays for the stats package. Technically, not needed,
    # but pandas Series / DataFrames sometimes throw warnings, so this way
    # we don't see them.
    col_a, col_b = tmp_df[col_a_name].to_numpy(), tmp_df[col_b_name].to_numpy()
    # Need to do a custom label because we (may) have two axes
    patch_a = mpatches.Patch(color=colors[0], alpha=0.5, label=f"{name_a} ({units_a})")
    patch_b = mpatches.Patch(color=colors[1], alpha=0.5, label=f"{name_b} ({units_b})")
    ax.legend(handles=[patch_a, patch_b])
    # Add Spearman and Pearson Correlation Coefficient values if we're supposed
    # to
    if corr:
        spearman_r, spearman_p = stats.spearmanr(col_a, col_b)
        pearson_r, pearson_p = stats.pearsonr(tmp_df[col_a_name], tmp_df[col_b_name])
        subtitle = f"Pearson R = {pearson_r:0.3f}, p = {printable_p(pearson_p)};"
        subtitle += f" Spearman R = {spearman_r:0.3f}, p = {printable_p(spearman_p)}\n"
        subtitle += f"From {mindate_str} to {maxdate_str}"
        pass
    if rolling:
        subtitle += f", {rolling}-Day Rolling Mean."
    if norm:
        subtitle += "\nValues Normalized to mean of 0.0 and Standard Dev of 1.0"
        ax.set_ylabel("Values Normalized to Mean 0.0, Std Dev 1.0")
    else:
        if ax == ax2:
            ax.set_ylabel(units_a)
        else:
            ax.set_ylabel(units_a)
            ax2.set_ylabel(units_b)
            pass
        pass
    # If we were not provided a title, make one up:
    if title is None:
        title = f"{name_a} ({units_a}) vs. {name_b} ({units_b}) in Philadelphia"
        pass
    # Append the subtitle to the title. Getting the title and subtitle to look
    # nice but with different font sizes turned out to be a pain, so I gave up
    # on that.
    title += "\n" + subtitle
    ax.set_title(title, fontsize=14)
    # Add the x-values label
    ax.set_xlabel("Date")
    # Make the plot "tight" because I think it looks better.
    plt.tight_layout()
    return


def plot_cols(df: pd.DataFrame, cols: list, title: str=None,
              norm: bool=False, subtitle: str="", figsize: Tuple[int, int]=(9,6)
              ) -> None:
    """
    Convenience function to plot one or more columns with dates.
    """
    # Make a copy so we don't clobber any data
    df = df.copy()
    # If it's just two, call the plot_two_ts() function in stead.
    if len(cols) == 2:
        return plot_two_ts(df, cols[0], cols[1], title, norm, subtitle)
    fig, ax = plt.subplots(figsize=figsize)
    # Get the minimum and maximum dates
    mindate, maxdate = df['date'].min(),df['date'].max()
    # Make the x-axis "tight" (no empty space before or after the actual data)
    ax.set_xlim(mindate,maxdate)
    # Make the mininum and maximum date "pretty"
    mindate_str, maxdate_str = mindate.strftime('%b %d, %Y'), maxdate.strftime('%b %d, %Y')
    labels = []
    for col in cols:
        if 'date' == col:
            continue
        name, units = human_col_info(col)
        label = f"{name} ({units})"
        labels.append(label)
        if norm:
            df[col] = normalize_to_mean(df[col])
            pass
        df.plot(x='date', y=col, ax=ax, lw=1.5, alpha=0.5)
        pass
    ax.legend(labels=labels)
    if norm:
        subtitle += "Normalized to mean of 0.0 and Standard Dev of 1.0"
        ax.set_ylabel("Values Normalized to Mean 0.0, Std Dev 1.0")
        pass
    title += "\n" + subtitle
    ax.set_title(title, fontsize=14)
    ax.set_xlabel("Date")
    plt.tight_layout()
    return


# Data

## Data Labels

In [ ]:
col_units = {
    'pm25_ugm3': r'${{\mu}g}/{m^3}$',
    'co_ppm': r'$ppm$',
    'o3_ppm': r'$ppm$',
    'no2_ppb': r'$ppb$',
    'so2_ppb': r'$ppb$',
    'temp_f_max': r'$°F$',
    'temp_f_mean': r'$°F$',
    'temp_f_min': r'$°F$',
    'dew_f_max': r'$°F$',
    'dew_f_mean': r'$°F$',
    'dew_f_min': r'$°F$',
    'humid_max': r'$Percent$',
    'humid_mean': r'$Percent$',
    'humid_min': r'$Percent$',
    'wind_mph_max': r'$mph$',
    'wind_mph_mean': r'$mph$',
    'wind_mph_min': r'$mph$',
    'mmhg_max': r'$mmHg$',
    'mmhg_mean': r'$mmHg$',
    'mmhg_min': r'$mmHg$',
    'precip_inches': r'$inches$',
    'temp_c_max': r'$°C$',
    'temp_c_mean': r'$°C$',
    'temp_c_min': r'$°C$',
    'dew_c_max': r'$°C$',
    'dew_c_mean': r'$°C$',
    'dew_c_min': r'$°C$',
    'wind_kph_max': r'$km/h$',
    'wind_kph_mean': r'$km/h$',
    'wind_kph_min': r'$km/h$',
    'precip_cm': r'$cm$',
    'temp_dew_diff_c': r'$°C$',
    'covid_hosp': r'$Number$',
    'day_of_week': f'$0-6$',
}

col_names = {
    'pm25_ugm3': r'Particulate Matter ≥ 2.5 $nm$',
    'co_ppm': r'Carbon Monoxide',
    'o3_ppm': 'Ozone',
    'no2_ppb': 'Nitrogen Dioxide',
    'so2_ppb': 'Sulfur Dioxide',
    'temp_f_max': r'Max Temperature',
    'temp_f_mean': r'Mean Temperature',
    'temp_f_min': r'Min Temperature',
    'dew_f_max': r'Max Dew Point',
    'dew_f_mean': r'Mean Dew Point',
    'dew_f_min': r'Min Dew Point',
    'humid_max': r'Max Relative Humidity',
    'humid_mean': r'Mean Relative Humidity',
    'humid_min': r'Min Relative Humidity',
    'wind_mph_max': r'Max Wind Speed',
    'wind_mph_mean': r'Mean Wind Speed',
    'wind_mph_min': r'Min Wind Speed',
    'mmhg_max': r'Max Atmospheric Pressure',
    'mmhg_mean': r'Mean Atmospheric Pressure',
    'mmhg_min': r'Min Atmospheric Pressure',
    'precip_inches': r'Precipitation',
    'temp_c_max': r'Max Temperature',
    'temp_c_mean': r'Mean Temperature',
    'temp_c_min': r'Min Temperature',
    'dew_c_max': r'Max Dew Point',
    'dew_c_mean': r'Mean Dew Point',
    'dew_c_min': r'Min Dew Point',
    'wind_kph_max': r'Max Wind Speed',
    'wind_kph_mean': r'Mean Wind Speed',
    'wind_kph_min': r'Min Wind Speed',
    'precip_cm': r'Precipitation',
    'temp_dew_diff_c': r'Mean Temerature Minus Mean Dew Point',
    'covid_hosp': r'COVID-19 Related Hospitalizations',
    'day_of_week': r'Day of Week as a Number',
}

## Daily Weather Data
The daily weather information was manually scraped from https://www.wunderground.com/history/daily/us/pa/philadelphia/KPHL/date/ on 2021-02-10 and stored in Google Drive as a CSV. For some reason there are two days which are missing any data from this website.

In [ ]:
# "Download" the weather data from the Google Sheet containing all the data for this project.
weather_df = get_google_sheet_df("1F5DWdR3tAGRSqoPfFwnrvXiZV_WHCbdfv3T_uGFjsiY","Philly Weather 2020")
weather_df.head()

In [ ]:
# Make sure the date column is a datetime
weather_df['date'] = pd.to_datetime(weather_df['date'], infer_datetime_format=True, errors='ignore')
# Include only data for 2019 or 2020
indexes = (weather_df['date']>=pd.Timestamp(2019,1,1)) & (weather_df['date']<pd.Timestamp(2021,1,1))
weather_df = weather_df[indexes]
weather_df['date_idx'] = weather_df['date']
weather_df.set_index('date_idx', inplace=True)
# Show information of the data we've downloaded.
weather_df.describe()

In [ ]:
# Add celsius data
for col in weather_df.columns:
    if "_f_" not in col:
        continue
    first, second = col.split("_f_")
    new_col = f"{first}_c_{second}"
    weather_df[new_col] = (weather_df[col] - 32) * (5/9)
    pass
# Add km/h data
for col in weather_df.columns:
    if "_mph_" not in col:
        continue
    first, second = col.split("_mph_")
    new_col = f"{first}_kph_{second}"
    weather_df[new_col] = weather_df[col] * 1.60934
    pass
weather_df['precip_cm'] = weather_df['precip_inches'] * 2.54

## Air Quality Data

Downloaded from https://www.epa.gov/outdoor-air-quality-data/download-daily-data manually converted to one aggregated CSV file.

In [ ]:
aq_df = get_google_sheet_df("1F5DWdR3tAGRSqoPfFwnrvXiZV_WHCbdfv3T_uGFjsiY","Philly AQ All 2020")

In [ ]:
aq_df.set_index('date', inplace=True)

In [ ]:
# aqs_param  units   
# PM2.5      ug/m3 LC    503
# O3         ppm         346
# CO         ppm         275
# NO2        ppb         274
# S02        ppb         270
def get_specific_df(df, aqs_param, new_col_name):
    """
    Create a DataFrame for a specific air quality measurement where there
    is zero or one reading per day by taking the average (mean) for those
    days with more than one reading.
    """
    # Get only the values we want.
    df = df[(aq_df['aqs_param'] == aqs_param)]
    # Only the one column, but keep it as a DataFrame not a Series
    df = df['value'].to_frame().dropna()
    # Rename the column
    df.rename(columns={'value':new_col_name}, inplace=True)
    # Remove non-numbers
    df.dropna(inplace=True)
    # Group by date and take the mean. Basically, if there is more than
    # one reading for that day, take the mean of all readings and just
    # keep the mean. This way, all days have one and only one "reading"
    df = df.groupby('date').mean()
    # Return this new DataFrame
    return df

def create_merged_air_quality_df(df):
    """
    Take the aq_df and create a df with columns that are the individual readings
    for an air quality measure and the rows are one row per day within the
    time frame included.
    """
    # Create a DataFrame with one PM2.5 reading per day which is an average of all
    # readings for that day
    pm25_df = get_specific_df(aq_df, 'PM2.5', 'pm25_ugm3')
    # Repeate for other readings
    co_df = get_specific_df(aq_df,'CO','co_ppm')
    o3_df = get_specific_df(aq_df,'O3','o3_ppm')
    no2_df = get_specific_df(aq_df,'NO2','no2_ppb')
    so2_df = get_specific_df(aq_df,'SO2','so2_ppb')
    # Merge them all together using the date index as the key
    aq2_df = pd.merge(pm25_df,co_df, how='outer', left_index=True, right_index=True)
    aq2_df = pd.merge(aq2_df,o3_df, how='outer', left_index=True, right_index=True)
    aq2_df = pd.merge(aq2_df,no2_df, how='outer', left_index=True, right_index=True)
    aq2_df = pd.merge(aq2_df,so2_df, how='outer', left_index=True, right_index=True)
    aq2_df.sort_index()
    aq2_df['date'] = pd.to_datetime(aq2_df.index)
    return aq2_df

aq2_df = create_merged_air_quality_df(aq_df)
aq2_df.describe()

## Merge the weather data with the air quality data

In [ ]:
# Merge the above into the add_df DataFrame so we can do some easy plotting
# and analysis.
all_df = pd.merge(aq2_df, weather_df, how='outer', left_index=True, right_index=True).sort_index()
all_df['date'] = pd.to_datetime(all_df.index, infer_datetime_format=True, errors='ignore')
all_df.describe()

## COVID-19 Related Data

Downloaded from https://phl.carto.com/api/v2/sql?filename=covid_hospitalizations_by_date&format=csv&skipfields=cartodb_id,the_geom,the_geom_webmercator&q=SELECT%20*%20FROM%20covid_hospitalizations_by_date

In [ ]:
covid_df = get_google_sheet_df("1F5DWdR3tAGRSqoPfFwnrvXiZV_WHCbdfv3T_uGFjsiY",'COVID-19 Hospitalizations')
covid_df = covid_df[['date','count']].groupby('date').sum()
covid_df.columns = ['covid_hosp']
covid_df.dropna(inplace=True)
covid_df['date'] = pd.to_datetime(covid_df.index, infer_datetime_format=True, errors='ignore')
covid_df

In [ ]:
plot_cols(covid_df, ['covid_hosp'], "COVID-19 Replated Hospitalizations")

In [ ]:
# The previous chart looked suspiciously like there might be a weekly pattern to reporting, so let's look at day of week information.

covid_df['day_of_week'] = covid_df['date'].dt.dayofweek
plot_two_ts(covid_df, 'covid_hosp', 'day_of_week', figsize=(19,12)) #, title="Number of Reported COVID-19 Hospitalizations vs Day of Week")

In [ ]:
# The previous chart looked suspiciously like there might be a weekly pattern to reporting, so let's look at day of week information.

covid_df['day_of_week'] = covid_df['date'].dt.dayofweek

mean = covid_df.groupby('day_of_week')['covid_hosp'].mean()
std = covid_df.groupby('day_of_week')['covid_hosp'].std()
fig, ax = plt.subplots(figsize=(9,6))
ax.errorbar(mean.index, mean, xerr=0.5, yerr=2*std, linestyle='')
ax.set_title("Mean Number of Reported Hospitalizations with 95% CI")

In [ ]:
# Let's see if we can factor out the weekday effect.
covid_df['covid_hosp_rolling_mean'] = covid_df['covid_hosp'].rolling(7).mean()
col_names['covid_hosp_rolling_mean'] = 'COVID-19 Reported Hosp. 7-Day Rolling Mean'
col_units['covid_hosp_rolling_mean'] = '$Number$'
covid_df['covid_hosp_rolling_diff'] = covid_df['covid_hosp'] - covid_df['covid_hosp_rolling_mean']
col_names['covid_hosp_rolling_diff'] = 'COVID-19 Reported Hosp. Minus 7-Day Rolling Mean'
col_units['covid_hosp_rolling_diff'] = '$Number$'
covid_df

In [ ]:
all_df = pd.merge(all_df,covid_df, how='outer', left_index=True, right_index=True)
all_df['date'] = pd.to_datetime(all_df.index, errors='ignore', infer_datetime_format=True)

In [ ]:
plot_two_ts(covid_df, 'covid_hosp_rolling_diff', 'day_of_week', title="Number of Reported COVID-19 Hospitalizations vs Day of Week\nAfter Subtracting Rolling Mean", figsize=(18,12))

In [ ]:
mean = covid_df.groupby('day_of_week')['covid_hosp_rolling_diff'].mean()
std = covid_df.groupby('day_of_week')['covid_hosp_rolling_diff'].std()
fig, ax = plt.subplots(figsize=(9,6))
ax.errorbar(mean.index, mean, xerr=0.5, yerr=2*std, linestyle='')
ax.set_title("Mean Number of Reported Hospitalizations with 95% CI")

# Exploratory Data Anaylsis (EDA)

## Check weather correlations
Just testing out some known correlations as a sanity check.

Here we plot the minimum and maximum temperatures over time. They should
look like they follow each other rather closely.

In [ ]:
plot_two_ts(all_df, 'temp_c_max', 'temp_c_min')

Now we try the same for the dewpoint which should also track pretty closely.

In [ ]:
plot_two_ts(all_df, 'dew_c_max', 'dew_c_min')

Lastly, let's check the mean temerature and mean dew point (which should track closely) along with the amount of precipitation, whcich may or may not be correlated with the above.

In [ ]:
plot_two_ts(all_df, 'dew_c_mean', 'temp_c_mean')

In [ ]:
plot_two_ts(all_df, 'dew_c_mean', 'precip_cm')

In [ ]:
# dew point minus min temperature
all_df['temp_dew_diff_c'] = all_df['temp_c_mean'] - all_df['dew_c_mean']
col_names['temp_dew_diff_c'] = 'Mean Temp Minus Mean Dew Pt'
col_units['temp_dew_diff_c'] = '$°C$'
plot_two_ts(all_df, 'temp_dew_diff_c', 'precip_cm')

In [ ]:
plot_two_ts(all_df[all_df['temp_c_min'] > 0.0], 'temp_c_mean', 'precip_cm', "Above Freezing Temperatures vs. Precipitation in Philadelphia")

In [ ]:
plot_two_ts(all_df, 'temp_c_mean', 'wind_kph_mean')

## Paring the Data Set Down to 2020 Only

For now, I am going to only look at 2020 data.

In [ ]:
indexes = (all_df['date']>=pd.Timestamp(2020,1,1)) & (all_df['date']<pd.Timestamp(2021,1,1))
all_df=all_df[indexes]

## Air Quality Plots

In [ ]:
plot_cols(aq2_df, [col for col in aq2_df.columns if 'date' != col],
          f"All Air Quality Measures in Philadelphia in Data Set",
          norm=True)

In [ ]:
plot_two_ts(aq2_df, 'pm25_ugm3','co_ppm', norm=True)

In [ ]:
plot_two_ts(aq2_df, 'pm25_ugm3','o3_ppm', norm=True)

In [ ]:
plot_two_ts(aq2_df, 'pm25_ugm3','no2_ppb', norm=True)

In [ ]:
plot_two_ts(aq2_df, 'co_ppm','o3_ppm', norm=True)

In [ ]:
plot_two_ts(aq2_df, 'co_ppm','no2_ppb', norm=True)

In [ ]:
plot_two_ts(aq2_df, 'co_ppm','no2_ppb')

In [ ]:
plot_two_ts(aq2_df, 'o3_ppm','no2_ppb', norm=True)

## Calculate multiple cross correlations.

I chose to use the [Spearman's rank correlation coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient) in stead of the more common [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) because I assumed that any correlations may not be strictly linear and the Wikipedia page on Spearman suggested that it might be more robust to non-linear correlations than the Pearson without increasing the liklihood of [spurious correlations](https://tylervigen.com/spurious-correlations).

### Everything v Everything

I first wanted to see a cross-correlation of everything to everything, both for curiosity and also a sanity check. I am initially only interested in the magnitude of the correlation (e.g., the absolute value), since I think that very negatively correlated columns are just as interesting as very positively correlated items.

I use the pandas built-in corr [function](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) which does this for me and then used the [seabrone heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html) method to display. I didn't find the default heatmap colors easy to see so I picked a different colormap.

In order to exclude "self-correlations", I set all correlation values of 1.0 to [numpy.nan](https://numpy.org/doc/stable/reference/constants.html) which eliminates them from the heatmap.

Luckily, even the converted columns had correlations of 1.0, so it all looked good.

In [ ]:
# Get the cross-correlation of all columns. I chose the Spearman rank order
# correlation coefficient instead of the default Pearson because the Spearman
# is less sensitive to non-linear correlations without significantly increasing
# false-positives and I believe that some of the correlations of interest may
# be non-linear. See Also:
# https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient
corrs = all_df.corr(method='spearman')
fig, ax = plt.subplots(figsize=(18,12))
sns.heatmap(corrs.replace(1.0, np.nan).abs(), annot=True, ax=ax, cmap="YlGnBu")
ax.set_title("Absolute Value of Spearman Rank Order Correlation Coefficients $≤ 1.0$")

### Most Things v Most Things

Since I really want to look at things which may have correlations but are not completely obvious (e.g., not min temperature vs. max temperature), and I don't need to cross correlate coverted columns, I remove all of the empirical columns and keep only the metric columns. After going over the values, I noted that all correlations above 0.8 were uninteresting (e.g., like the min and max temperature), so I want to plot only values ≤ 0.8. This still leaves a couple non-interesting correlations (e.g., min and max humidity or min and max dew points) but it leaves some interesting ones which are surprisingly strongly correlated (e.g., temperature and COVID-19 hospitalizations)

In [ ]:
emperical_columns = []
for x in all_df.columns:
    if 'inch' in x or 'mph' in x or '_f_' in x:
        emperical_columns.append(x)
        pass
    pass
tmp_df = all_df.drop(columns=emperical_columns + ['so2_ppb']).corr(method='spearman')
labels = []
mapper = {}
max_corr = 0.8
for col in tmp_df.columns:
    name, units = human_col_info(col)
    units = units.replace('$','').replace('{','').replace('}','').replace('\\','')
    label = f"{name} ({units})"
    # print(f"col -> '{label}'")
    labels.append(label)
    mapper[col] = label
    pass
tmp_df.columns = labels
tmp_df.rename(mapper, inplace=True)
fig, ax = plt.subplots(figsize=(18,12))
tmp_df[tmp_df.abs() > max_corr] = np.nan
sns.heatmap(tmp_df.abs(), annot=True, ax=ax, cmap="YlGnBu")
ax.set_title(f"Absolute Value of Spearman Rank Order Correlation Coefficients $≤ {max_corr}$")
# ax.set_xlabel(labels)
# ax.set_ylabel(labels)

### Looking Deeper at Temperature v COVID-19 Hospitalizations

We found a strong negative correlation (Spearman R = $-0.770$, p = $6.1\times10^{-58}$; Pearson R = $-0.753$, p = $3.7\times10^{-54}$) between temperature and the 7-day rolling mean for the reported number of COVID-19 related hospitalizations. This is somewhat consistent with published findings (Xie 2020).

**Citations**

Xie, J., & Zhu, Y. (2020). Association between ambient temperature and COVID-19 infection in 122 cities from China. Science of the Total Environment, 724, 138201. [link](https://doi.org/10.1016/j.scitotenv.2020.138201)

In [ ]:
plot_two_ts(all_df, 'temp_c_mean', 'covid_hosp_rolling_mean', figsize=(18,12))

#### Contradicting other findings.

However, Zoran et al. (Zoran 2020) found a positive correlation between ground-level ozone and COVID-19 infections, wereas I found a modest negative correlation (Spearman R = $-0.395$, p = $1.5\times10^{-11}$, Pearson R = $-0.452$, p = $4.7\times10^{-15}$).

Similarly, I found that $NO_2$ was positively correlated with COVID-19 hospitalizations (Spearman R = $-0.131$, p = $0.06$ [not significant], Pearson R = $-0.299$, p = $1.8\times10^{-5}$), which is the opposite of what was reported by Zoran et al. I was also unable to validate (not significant) the negative relationship with humidity or precipitation while again I found the opposite correlation with wind (Spearman R = $-0.202$, p = $5.4\times10^{-4}$, Pearson R = $-0.210$, p = $3.2\times10^{-4}$).

**Citations**

Zoran, M. A., Savastru, R. S., Savastru, D. M., & Tautan, M. N. (2020). Assessing the relationship between ground levels of ozone (O3) and nitrogen dioxide (NO2) with coronavirus (COVID-19) in Milan, Italy. Science of The Total Environment, 740, 140005. [link](https://doi.org/10.1016/j.scitotenv.2020.140005)

In [ ]:
plot_two_ts(all_df, 'o3_ppm', 'covid_hosp_rolling_mean')
plot_two_ts(all_df, 'no2_ppb', 'covid_hosp_rolling_mean')
plot_two_ts(all_df, 'humid_mean', 'covid_hosp_rolling_mean')
plot_two_ts(all_df, 'wind_kph_mean', 'covid_hosp_rolling_mean')
plot_two_ts(all_df, 'precip_cm', 'covid_hosp_rolling_mean')


In [ ]:
# List the things that most correlate with particulate matter in the air.
# We use the absolute value because we're only interested in the magnitude
# and we want only values below 0.9 because the other pm25 columnes have
# correlations above 0.9 but nothing else does.
corrs['pm25_ugm3'].abs()[corrs['pm25_ugm3'].abs() < 0.9].sort_values()[-10:]
# From the above output, we see that values most correlated with particulate
# matter in the air are the average (mean) wind speeds and the minimum recorded
# wind speed, which makes sense since mininum wind speed is more strongly
# correlated with average wind speed than maximum wind speed (think gusts)
# interestingly, precipitation seems to have almost no correlation with
# particulate matter in the air, which I did not expect, even though
# humidity had a moderate correlation.

In [ ]:
# One of the issues is that the definition of "mean" wind speed was not well
# defined at the source. The wind_mph_mean and wind_kmh_mean columns are clearly
# not the means of the min and max, but I'm not sure how they are calculates
# (e.g., sampled each hour? minute? second? then over the day's 24 hours?)
# and I worried that the "spikiness" (e.g., variance) of the data might make
# detecting a "real" correlation more challenging. Assuming that on most days
# the real particulate matter content across the entier greater Philadelphia
# area varies less bruskly than once-or-twice-a-day readings at one or two
# locations (the readings are irregular), I believe it is appropriate to consider
# a 7-day smoothed average to better represent the "real" particulate matter
# air content for the area, so I rolled up the values to 7-day averages here:
rolling_df = all_df.rolling(7).mean()
rolling_df

In [ ]:
# Now I recalculate the Spearman cross-corralations on the moving average data
rolling_corrs = rolling_df.corr(method='spearman')
rolling_corrs

In [ ]:
rolling_df['date'] = pd.to_datetime(rolling_df.index)

### Air Borne Particulate Matter v Wind Speed



**Citations**

Jones, A. M., Harrison, R. M., & Baker, J. (2010). The wind speed dependence of the concentrations of airborne particulate matter and NOx. Atmospheric Environment, 44(13), 1682-1690.

Zhang, B., Jiao, L., Xu, G., Zhao, S., Tang, X., Zhou, Y., & Gong, C. (2018). Influences of wind and precipitation on different-sized particulate matter concentrations (PM 2.5, PM 10, PM 2.5–10). Meteorology and Atmospheric Physics, 130(3), 383-392.

In [ ]:
plot_two_ts(all_df, 'pm25_ugm3','wind_kph_mean')

In [ ]:
plot_two_ts(all_df, 'pm25_ugm3','wind_kph_mean', rolling=7)

In [ ]:
# And repeat for the PM2.5 column as before to find the largest magnitude
# correlations:
for col in "pm25_ugm3	co_ppm	o3_ppm	no2_ppb".split():
    print(f"---- Spearman R Values with {col}")
    print(rolling_corrs[col].abs()[rolling_corrs[col].abs() < 0.999].sort_values()[-10:])
    pass

### Ozone v Temp Known Correlations
Here I check if I see what has been generally established - namely that ozone increases with temperature. I do find reasonably strong correlations (Pearson R = 0.66, Spearman R = 0.69) that are highly significant (p << 0.00000000001)

**Note:** It is surprising to me that the ozone levels seem to anticipate the rise in temperature on the rolling-mean plot.

**Citations:**

Barnett, J. J., Houghton, J. T., & Pyle, J. A. (1975). The temperature dependence of the ozone concentration near the stratopause. Quarterly Journal of the Royal Meteorological Society, 101(428), 245-257.

Olszyna, K. J., Luria, M., & Meagher, J. F. (1997). The correlation of temperature and rural ozone levels in southeastern USA. Atmospheric environment, 31(18), 3011-3022.

Pusede, S. E., Steiner, A. L., & Cohen, R. C. (2015). Temperature and recent trends in the chemistry of continental surface ozone. Chemical reviews, 115(10), 3898-3918.

In [ ]:
plot_two_ts(rolling_df, 'o3_ppm','temp_c_mean', rolling=7)

### NO2 v Temp

Likewise, I check to see if NO2 is inversely correlated with temperature as established in the literature. The inverse correlations exist (Pearson R = -0.79, Spearman R = -0.79) both highly significant.

**Citations**

Schindlbacher, A., Zechmeister‐Boltenstern, S., & Butterbach‐Bahl, K. (2004). Effects of soil moisture and temperature on NO, NO2, and N2O emissions from European forest soils. Journal of Geophysical Research: Atmospheres, 109(D17).

Goldberg, D. L., Anenberg, S. C., Kerr, G. H., Mohegh, A., Lu, Z., & Streets, D. G. (2021). TROPOMI NO2 in the United States: A detailed look at the annual averages, weekly cycles, effects of temperature, and correlation with surface NO2 concentrations. Earth's Future, e2020EF001665.

In [ ]:
plot_two_ts(rolling_df, 'no2_ppb','temp_c_mean', rolling=7)